In [2]:
from trackml.dataset import load_event
from trackml.dataset import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random


%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from tqdm import tqdm
from sklearn.neighbors import KDTree

In [3]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_5.zip'):
    if cnt == 1:
        break
    cnt += 1

In [4]:
	def extend(submission,hits):

		df = submission.merge(hits,  on=['hit_id'], how='left')
		df = df.assign(d = np.sqrt( df.x**2 + df.y**2 + df.z**2 ))
		df = df.assign(r = np.sqrt( df.x**2 + df.y**2))
		df = df.assign(arctan2 = np.arctan2(df.z, df.r))

		for angle in range(-180,180,1):

		    #print ('\r %f'%angle, end='',flush=True)
		    #df1 = df.loc[(df.arctan2>(angle-0.5)/180*np.pi) & (df.arctan2<(angle+0.5)/180*np.pi)]
		    df1 = df.loc[(df.arctan2>(angle-1.0)/180*np.pi) & (df.arctan2<(angle+1.0)/180*np.pi)]

		    min_num_neighbours = len(df1)
		    if min_num_neighbours<4: continue

		    hit_ids = df1.hit_id.values
		    x,y,z = df1.as_matrix(columns=['x', 'y', 'z']).T
		    r  = (x**2 + y**2)**0.5
		    r  = r/1000
		    a  = np.arctan2(y,x)
		    tree = KDTree(np.column_stack([a,r]), metric='euclidean')

		    track_ids = list(df1.track_id.unique())
		    num_track_ids = len(track_ids)
		    min_length=3

		    for i in range(num_track_ids):
		        p = track_ids[i]
		        if p==0: continue

		        idx = np.where(df1.track_id==p)[0]
		        if len(idx)<min_length: continue

		        if angle>0:
		            idx = idx[np.argsort( z[idx])]
		        else:
		            idx = idx[np.argsort(-z[idx])]


		        ## start and end points  ##
		        idx0,idx1 = idx[0],idx[-1]
		        a0 = a[idx0]
		        a1 = a[idx1]
		        r0 = r[idx0]
		        r1 = r[idx1]

		        da0 = a[idx[1]] - a[idx[0]]  #direction
		        dr0 = r[idx[1]] - r[idx[0]]
		        direction0 = np.arctan2(dr0,da0) 

		        da1 = a[idx[-1]] - a[idx[-2]]
		        dr1 = r[idx[-1]] - r[idx[-2]]
		        direction1 = np.arctan2(dr1,da1) 

	 
		        ## extend start point
		        ns = tree.query([[a0,r0]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)
		        direction = np.arctan2(r0-r[ns],a0-a[ns])
		        ns = ns[(r0-r[ns]>0.01) &(np.fabs(direction-direction0)<0.04)]
	   
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p 

		        ## extend end point
		        ns = tree.query([[a1,r1]], k=min(20,min_num_neighbours), return_distance=False)
		        ns = np.concatenate(ns)

		        direction = np.arctan2(r[ns]-r1,a[ns]-a1)
		        ns = ns[(r[ns]-r1>0.01) &(np.fabs(direction-direction1)<0.04)] 
		        
		        for n in ns:
		            df.loc[ df.hit_id==hit_ids[n],'track_id' ] = p
		#print ('\r')
		df = df[['event_id', 'hit_id', 'track_id']]
		return df


In [5]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [6]:
def merge(cl1, cl2, min_cnt): # merge cluster 2 to cluster 1
    d = pd.DataFrame(data={'s1':cl1,'s2':cl2})
    d['N1'] = d.groupby('s1')['s1'].transform('count')
    d['N2'] = d.groupby('s2')['s2'].transform('count')
    maxs1 = d['s1'].max()
    cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) &  (d['N2'].values>min_cnt))
    #cond = np.where((d['N2'].values>d['N1'].values) & (d['N2'].values<20) )

    s1 = d['s1'].values 
    s1[cond] = d['s2'].values[cond]+maxs1 
    return s1

In [367]:
z_result = np.load('predicts/200_zshift.npy')
z_result = z_result[:, 0, :]

result_53 = np.load('predicts/53_all.npy')
result_53 = np.array(result_53)

luis = np.load('predicts/luis_all.npy')

In [368]:
result_53.shape

(600, 100287)

In [369]:
z_result.shape

(201, 100287)

In [370]:
luis.shape

(338, 100287)

In [7]:
result = []
for i in range(-100,101):
    result += [np.load('predicts/53/{}.npy'.format(i))]

result = np.array(result)
result = result[:, 0, :]

In [9]:
#result = result_53
labels = range(result.shape[1])

#for k in [18, 16, 14, 12, 10, 8, 6, 4, 0]:
for k in [0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))

0.04607091374792327


In [ ]:
0.547577784065302 result_53
0.4912658962859528 z_result
0.5071140015567802 luis

In [378]:
result = np.vstack([z_result, luis])
np.random.shuffle(result)

In [362]:
#result = result_53
labels = range(result.shape[1])

for k in [18,16,14, 12, 10, 8, 6, 4, 0]:
#for k in [15,10,5, 0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))
    #submission = extend(submission,hits)
    #submission = extend(submission,hits)

    print(score_event(truth, submission))
    
    labels = submission['track_id'].values
   

0.02946572701478705
0.02946572701478705
0.05727761228580784
0.05727761228580784
0.11534065410312926
0.11534065410312926
0.23344574246875743
0.23344574246875743
0.36729231106049864
0.36729231106049864
0.45506391156244447
0.45506391156244447
0.4944932151899727
0.4944932151899727
0.5049299479307393
0.5049299479307393
0.5073244085906707
0.5073244085906707


In [329]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5848926941183028


In [330]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5850779423213555


In [325]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5850817906028537


In [326]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5853420574657813


In [327]:
submission = extend(submission,hits)
print(score_event(truth, submission))

0.5855217023763549


In [352]:
labels1 = merge(labels_luis, labels, 0)
submission = create_one_event_submission(0, hits['hit_id'].values, labels1)
print(score_event(truth, submission))

0.4913849820080143


In [ ]:
0.5076282691673919
0.5067205748968073
0.4731231005334312

In [ ]:
0.4912658962859528

In [ ]:
result = result_53
labels = range(z_result.shape[1])

for k in [15,10,5, 0]:
    for i in range(len(result)):
        labels = merge(labels, result[i], k)
  
    submission = create_one_event_submission(0, hits['hit_id'].values, labels)
    print(score_event(truth, submission))
    submission = extend(submission,hits)
    print(score_event(truth, submission))
    
    labels = submission['track_id'].values
   

In [303]:
submission = create_one_event_submission(0, hits['hit_id'].values, labels)

In [307]:
score_event(truth, ext_sub)

0.5102744886009511

In [ ]:
0.5855217023763549
0.5853420574657813
0.5850817906028537
0.5849290849426552
0.5827447401532195
0.5810983311646127
0.5807878755572605
0.5792614621063498
0.5102744886009511